## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# Files to load
city_data = "../WeatherPy_Database.csv"


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(city_data)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Bubaque,GW,11.28,-15.83,78.49,72,0,14.23,clear sky,2020-12-29 01:00:25
1,1,Turbat,PK,26.00,63.04,54.16,32,0,13.33,clear sky,2020-12-29 01:00:25
2,2,Rikitea,PF,-23.12,-134.97,78.62,78,100,16.58,moderate rain,2020-12-29 00:56:34
3,3,Mataura,NZ,-46.19,168.86,76.32,40,45,3.76,scattered clouds,2020-12-29 00:56:34
4,4,Hilo,US,19.73,-155.09,82.40,58,20,9.17,few clouds,2020-12-29 01:00:09


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Bubaque,GW,11.28,-15.83,78.49,72,0,14.23,clear sky,2020-12-29 01:00:25
1,1,Turbat,PK,26.00,63.04,54.16,32,0,13.33,clear sky,2020-12-29 01:00:25
2,2,Rikitea,PF,-23.12,-134.97,78.62,78,100,16.58,moderate rain,2020-12-29 00:56:34
3,3,Mataura,NZ,-46.19,168.86,76.32,40,45,3.76,scattered clouds,2020-12-29 00:56:34
4,4,Hilo,US,19.73,-155.09,82.40,58,20,9.17,few clouds,2020-12-29 01:00:09
5,5,Te Anau,NZ,-45.42,167.72,70.41,35,100,3.60,overcast clouds,2020-12-29 01:00:26
6,6,Vaini,TO,-21.20,-175.20,84.20,74,75,4.70,moderate rain,2020-12-29 01:00:26
7,7,Jacmel,HT,18.23,-72.53,77.00,88,75,4.97,light rain,2020-12-29 01:00:27
8,8,Trinidad,BO,-14.83,-64.90,73.49,96,100,3.15,overcast clouds,2020-12-29 01:00:27
10,10,Cidreira,BR,-30.18,-50.21,73.29,82,57,15.30,broken clouds,2020-12-29 01:00:28


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                431
City                   431
Country                426
Lat                    431
Lng                    431
Max Temp               431
Humidity               431
Cloudiness             431
Wind Speed             431
Current Description    431
Date                   431
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
clean_df.count()

City_ID                426
City                   426
Country                426
Lat                    426
Lng                    426
Max Temp               426
Humidity               426
Cloudiness             426
Wind Speed             426
Current Description    426
Date                   426
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bubaque,GW,78.49,clear sky,11.28,-15.83,
1,Turbat,PK,54.16,clear sky,26.00,63.04,
2,Rikitea,PF,78.62,moderate rain,-23.12,-134.97,
3,Mataura,NZ,76.32,scattered clouds,-46.19,168.86,
4,Hilo,US,82.40,few clouds,19.73,-155.09,
5,Te Anau,NZ,70.41,overcast clouds,-45.42,167.72,
6,Vaini,TO,84.20,moderate rain,-21.20,-175.20,
7,Jacmel,HT,77.00,light rain,18.23,-72.53,
8,Trinidad,BO,73.49,overcast clouds,-14.83,-64.90,
10,Cidreira,BR,73.29,broken clouds,-30.18,-50.21,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")  

 
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [11]:
hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bubaque,GW,78.49,clear sky,11.28,-15.83,"Casa Dora, Bubaque"
1,Turbat,PK,54.16,clear sky,26.00,63.04,Mubashar Quarters
2,Rikitea,PF,78.62,moderate rain,-23.12,-134.97,Pension Maro'i
3,Mataura,NZ,76.32,scattered clouds,-46.19,168.86,Ellie's Villa
4,Hilo,US,82.40,few clouds,19.73,-155.09,Hilo Hawaiian Hotel


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]

In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))